## Create subsamples of validation cut

In [ ]:
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm

def get_stratified_sample(size, info):
    return train_test_split(info, train_size=size, test_size=None, stratify=info.SP)[0].sort_index()

def generate_subsamples(sp):
    for size in tqdm([1000, 10000]):
        info = pd.read_csv('./cut/{size}/M4-info.csv'.format(size=size))
        sp_info = info[info.SP == sp]
        train = pd.read_csv('./cut/full/val/train/{sp}.csv'.format(sp=sp), index_col=0)
        test = pd.read_csv('./cut/full/val/test/{sp}.csv'.format(sp=sp))
        test.index = train.index
        assert len(train) == len(test)
        
        os.makedirs('./cut/{size}/val/train'.format(size=size), exist_ok=True)
        os.makedirs('./cut/{size}/val/test'.format(size=size), exist_ok=True)
        train.loc[sp_info.M4id].to_csv('./cut/{size}/val/train/{sp}.csv'.format(size=size, sp=sp))
        test.loc[sp_info.M4id].to_csv('./cut/{size}/val/test/{sp}.csv'.format(size=size, sp=sp), index=False)
        
for sp in tqdm(['Daily', 'Hourly', 'Monthly', 'Quarterly', 'Weekly', 'Yearly']):
    generate_subsamples(sp)
    

## Drop unused index column in test cut train and test sets, M4-info files

In [ ]:
# for size in [1000, 10000]:
#     for sp in tqdm(['Daily', 'Hourly', 'Monthly', 'Quarterly', 'Weekly', 'Yearly']):
#         p = './cut/{size}'.format(size=size)
#         d = pd.read_csv(p + '/train/{sp}.csv'.format(sp=sp), index_col=False)
#         d.to_csv(p + '/train/{sp}.csv'.format(sp=sp), index=False)
        
# for size in [1000, 10000]:
#     for sp in tqdm(['Daily', 'Hourly', 'Monthly', 'Quarterly', 'Weekly', 'Yearly']):
#         p = './cut/{size}'.format(size=size)
#         d = pd.read_csv(p + '/test/{sp}.csv'.format(sp=sp), index_col=False)
#         d.to_csv(p + '/test/{sp}.csv'.format(sp=sp), index=False)

# for size in [1000, 10000]:
#     info = pd.read_csv('./cut/{size}/M4-info.csv'.format(size=size), index_col=0)
#     info.to_csv('./cut/{size}/M4-info.csv'.format(size=size), index=False)


## Check correctness

In [ ]:
import math
import numpy as np

def count_nonnan(row):
    return sum(1 for c in row if not math.isnan(c))

def count_nonnans(df):
    return df.apply(count_nonnan, axis=1).values

for sp in tqdm(['Daily', 'Hourly', 'Monthly', 'Quarterly', 'Weekly', 'Yearly']):
    for size in tqdm([1000, 10000, 'full']):
        p = './cut/{size}'.format(size=size)
        if size == 'full':
            info = pd.read_csv('./data/M4-info.csv')
        else:
            info = pd.read_csv(p + '/M4-info.csv')
            
        sp_info = info[info.SP == sp]
        
        ts = pd.read_csv('./data/{sp}-train.csv'.format(sp=sp), index_col=0)
        ts = ts.loc[sp_info.M4id]
        
        ts_val_train = pd.read_csv(p + '/val/train/{sp}.csv'.format(sp=sp), index_col=0)
        ts_val_test = pd.read_csv(p + '/val/test/{sp}.csv'.format(sp=sp), index_col=False)
        ts_test_train = pd.read_csv(p + '/train/{sp}.csv'.format(sp=sp), index_col=0)
        ts_test_test = pd.read_csv(p + '/test/{sp}.csv'.format(sp=sp), index_col=False)
        
        ######## TRAIN TEST CUT ########
        # length of train is correct
        ts_test_train_nonnans = count_nonnans(ts_test_train)
        ts_nonnans = count_nonnans(ts)
        assert (ts_test_train_nonnans == (ts_nonnans - sp_info.Horizon.values)).all()
        
        # first elements of train and original are equal
        assert np.allclose(ts.iloc[:, 0].values, ts_test_train.iloc[:, 0].values)
        
        # last nonnan elements of train are correct
        last_nonnan_idx = ts_test_train_nonnans - 1
        assert np.allclose(ts.values[np.arange(len(ts)), last_nonnan_idx],
                ts_test_train.values[np.arange(len(ts_test_train)), last_nonnan_idx])
        
        # first nan elements of train are nan
        assert np.isnan(ts_test_train.values[np.arange(len(ts_test_train)), last_nonnan_idx + 1]).all()
        
        # first nan elements of train are present in original
        assert not np.isnan(ts.values[np.arange(len(ts)), last_nonnan_idx + 1]).any()
        
        # number of test instance equals the horizon
        assert (count_nonnans(ts_test_test) == sp_info.Horizon.values).all()
        
        ######## VALIDATION CUT ########
        
        # Length of validation train is correct
        ts_val_train_nonnans = count_nonnans(ts_val_train)
        assert (ts_val_train_nonnans == (ts_nonnans - 2 * sp_info.Horizon.values)).all()
        
        # first elements of validation train and original are equal
        assert np.logical_or(np.isnan(ts_val_train.iloc[:, 0].values),
                             np.isclose(ts.iloc[:, 0].values, ts_val_train.iloc[:, 0].values)).all()
        
        # last nonnan elements of validation train are correct
        last_nonnan_idx = ts_val_train_nonnans - 1
        assert np.logical_or(
            np.isnan(ts_val_train.values[np.arange(len(ts_val_train)), last_nonnan_idx]),
            np.isclose(ts.values[np.arange(len(ts)), last_nonnan_idx],
                       ts_val_train.values[np.arange(len(ts_val_train)), last_nonnan_idx])
        ).all()
        
        # first nan elements of validation train are nan
        assert np.isnan(ts_val_train.values[np.arange(len(ts_val_train)), last_nonnan_idx + 1]).all()
        
        # first nan elements of val_train are present in test_train
        assert not np.isnan(ts_test_train.values[np.arange(len(ts_test_train)), last_nonnan_idx + 1]).any()
        
        # number of validation test instances equals the horizon
        assert (count_nonnans(ts_val_test) == sp_info.Horizon.values).all()